In [1]:
#import necessary dependecies
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import os
import warnings
import numpy as np  
import seaborn as sns
import pandas as pd, os, gc
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler, RobustScaler
%matplotlib inline
warnings.filterwarnings('ignore')

In [3]:
#import datasets
train = pd.read_csv('/home/felix/Downloads/zimnat/Train.csv')
test = pd.read_csv('/home/felix/Downloads/zimnat/Test.csv')
submission = pd.read_csv('/home/felix/Downloads/zimnat/SampleSubmission.csv')

In [4]:
#print all dataset shape
print(train.shape, test.shape,submission.shape)

(29132, 29) (10000, 29) (210000, 2)


In [5]:
#Store Products ID in Multidimensional array
products =train[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]
products_t =test[['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']]

In [6]:
#Axis Transformation
train = train.melt(id_vars=train.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )
test = test.melt(id_vars=test.columns[:8], value_vars=products, var_name = "PCODE", value_name="Label" )

In [7]:
#A merger
train['combiner']='x'

In [8]:
#Set
data=pd.concat([train,test],sort=False).reset_index(drop=True)

In [9]:
#preview random 3 sample
data.sample(3)

ID  join_date sex marital_status  birth_year branch_code  \
433602  PXZH8MX   1/6/2018   M              M        1998        30H5   
786802  OEPXKPU  1/12/2017   M              M        1991        748L   
524046  JP0GCSO   1/2/2019   M              M        1990        748L   

       occupation_code occupation_category_code PCODE  Label combiner  
433602            UJ5T                     T4MS  K6QO      1        x  
786802            K5GV                     T4MS  JZ9D      0      NaN  
524046            WMTK                     90QI  JZ9D      0        x

In [10]:
#Confirm Distribution (TEST AND SAMPLE SUBMISSION FILE)
print('==' * 18);print('TRAIN LABEL DISTRIBUTION');print('==' * 18);print(train['Label'].value_counts())
print('==' * 18);print('TEST LABEL DISTRIBUTION');print('==' * 18);print(test['Label'].value_counts())
print('==' * 18);print('SAMPLE SUBMISSION LABEL DISTRIBUTION');print('==' * 18);print(submission['Label'].value_counts())

TRAIN LABEL DISTRIBUTION
0    545419
1     66353
Name: Label, dtype: int64
TEST LABEL DISTRIBUTION
0    197147
1     12853
Name: Label, dtype: int64
SAMPLE SUBMISSION LABEL DISTRIBUTION
0    197147
1     12853
Name: Label, dtype: int64


In [11]:
train=data[data.combiner.notnull()].reset_index(drop=True);test=data[data.combiner.isna()].reset_index(drop=True)
train.drop('combiner', inplace=True, axis=1);test.drop(['Label','combiner'], inplace=True, axis=1)
print(train.shape, test.shape, submission.shape)

(611772, 10) (210000, 9) (210000, 2)


In [13]:
#Concatenate new column towards submission requirement
train['ID X PCODE'] = train['ID'] + ' X ' + train['PCODE'];test['ID X PCODE'] = test['ID'] + ' X ' + test['PCODE']

In [14]:
#Re-arrange column (not necessary)
train=train[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE','ID X PCODE',  'Label']]
test=test[['ID', 'join_date', 'sex', 'marital_status', 'birth_year', 'branch_code',
       'occupation_code', 'occupation_category_code', 'PCODE','ID X PCODE']]

In [15]:
#Lets Check for data co-existence in Train/Test
for col in ['PCODE','occupation_code', 'branch_code','occupation_category_code']:
    train_col = train[col].unique();test_col = test[col].unique()
    print ('Not in Test ' + col + ' ',[i  for i in train_col if i not in test_col])
    print ('Not in Train ' + col + ' ',[i  for i in test_col  if i not in train_col]);print('==' * 30)

Not in Test PCODE  []
Not in Train PCODE  []
Not in Test occupation_code  ['IE90', 'Q0LY', 'JSAX', '6XXU', '8HRZ', 'OQMY', 'INEJ', 'VZN9', 'UC7E', 'PSUY', 'WSRG', 'JQH3', 'LGTN', '738L', 'QQUP', '2XZ1', 'CAAV', 'LLLH', 'W1X2', 'DHSN', 'IX8T', '2US6', 'ZWPL', 'MEFQ', '9B5B', 'JUIP', 'BFD1', 'A4ZC', 'IMHI', 'E5PF', 'GZA8', '3YQ1', 'PJR4', 'NDL9', 'PPNK', '2686', '5LNN', '374O', 'URYD', 'M0WG', 'KBWO', 'ONY7', 'VYSA', 'KUPK', 'R7GL', 'HSVE', 'BER4', '6SKY', 'RH2K', 'ZHC2', 'W3ZV', 'FLXH', 'UYDZ', 'YJXM', '59QM']
Not in Train occupation_code  ['0ZND', '8CHJ', '9F96', 'HSI5', '93OJ', 'BIA0', 'E2MJ', 'JBJP', '0FOI']
Not in Test branch_code  []
Not in Train branch_code  []
Not in Test occupation_category_code  []
Not in Train occupation_category_code  []


In [16]:
train.head()

ID join_date sex marital_status  birth_year branch_code  \
0  4WKQSBB  1/2/2019   F              M        1987        1X1H   
1  CP5S02H  1/6/2019   F              M        1981        UAOD   
2  2YKDILJ  1/6/2013   M              U        1991        748L   
3  2S9E81J  1/8/2019   M              M        1990        1X1H   
4  BHDYVFT  1/8/2019   M              M        1990        748L   

  occupation_code occupation_category_code PCODE      ID X PCODE  Label  
0            2A7I                     T4MS  P5DA  4WKQSBB X P5DA      0  
1            2A7I                     T4MS  P5DA  CP5S02H X P5DA      0  
2            QZYX                     90QI  P5DA  2YKDILJ X P5DA      0  
3            BP09                     56SI  P5DA  2S9E81J X P5DA      0  
4            NO3L                     T4MS  P5DA  BHDYVFT X P5DA      0

In [17]:
test.head()

ID  join_date sex marital_status  birth_year branch_code  \
0  F86J5PC  1/12/2018   M              M        1984        94KC   
1  H6141K3  1/10/2019   M              M        1996        1X1H   
2  RBAYUXZ   1/1/2020   F              W        1968        UAOD   
3  KCBILBQ   1/2/2019   M              M        1989        94KC   
4  LSEC1ZJ   1/2/2020   F              M        1982        UAOD   

  occupation_code occupation_category_code PCODE      ID X PCODE  
0            DZRV                     90QI  P5DA  F86J5PC X P5DA  
1            J9SY                     90QI  P5DA  H6141K3 X P5DA  
2            2A7I                     T4MS  P5DA  RBAYUXZ X P5DA  
3            2A7I                     T4MS  P5DA  KCBILBQ X P5DA  
4            0KID                     T4MS  P5DA  LSEC1ZJ X P5DA

In [18]:
# LABEL ENCODE
def encode_LE(train,test,cols,verbose=True):
    for col in cols:
        df_comb = pd.concat([train[col],test[col]],axis=0)
        df_comb,_ = df_comb.factorize(sort=True)
        nm = col
        if df_comb.max()>32000: 
            train[nm] = df_comb[:len(train)].astype('int32')
            test[nm] = df_comb[len(train):].astype('int32')
        else:
            train[nm] = df_comb[:len(train)].astype('int16')
            test[nm] = df_comb[len(train):].astype('int16')
        del df_comb; x=gc.collect()
        if verbose: print(nm,', ',end='')

In [19]:
encode_LE(train, test, ['ID','branch_code', 'occupation_code','occupation_category_code','PCODE','sex','marital_status'])

ID , branch_code , occupation_code , occupation_category_code , PCODE , sex , marital_status , 

In [20]:
train.head()

ID join_date  sex  marital_status  birth_year  branch_code  \
0   5384  1/2/2019    0               2        1987            0   
1  13834  1/6/2019    0               2        1981           11   
2   3217  1/6/2013    1               6        1991            3   
3   3010  1/8/2019    1               2        1990            0   
4  12546  1/8/2019    1               2        1990            3   

   occupation_code  occupation_category_code  PCODE      ID X PCODE  Label  
0               19                         5     15  4WKQSBB X P5DA      0  
1               19                         5     15  CP5S02H X P5DA      0  
2              185                         1     15  2YKDILJ X P5DA      0  
3               82                         0     15  2S9E81J X P5DA      0  
4              157                         5     15  BHDYVFT X P5DA      0

In [21]:
test.head()

ID  join_date  sex  marital_status  birth_year  branch_code  \
0  16566  1/12/2018    1               2        1984            4   
1  18664  1/10/2019    1               2        1996            0   
2  29677   1/1/2020    0               7        1968           11   
3  22120   1/2/2019    1               2        1989            4   
4  23700   1/2/2020    0               2        1982           11   

   occupation_code  occupation_category_code  PCODE      ID X PCODE  
0               96                         1     15  F86J5PC X P5DA  
1              128                         1     15  H6141K3 X P5DA  
2               19                         5     15  RBAYUXZ X P5DA  
3               19                         5     15  KCBILBQ X P5DA  
4                3                         5     15  LSEC1ZJ X P5DA

# TRAIN AND TEST FILE PREPARED

## READ THE INSTRUCTION CAREFULLY AND CRACK THE REST FOR PREDICTION